# COMP3361 Part 1: Building a Transformer Encoder

Note: You should finish your code solution of Part 1 & 2 with A2p12.tgz. For Q2 & Q3, you should include your writeup in this notebook.

## Q2:

## Q3:

# COMP3361 Part 3: Generation with Large Language Model

## Load model and tokenizer

In this section, we will use [CodeLlama-7B](https://huggingface.co/codellama/CodeLlama-7b-hf) as the language model.

In [1]:
!pip install transformers datasets evaluate accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any
import os
import json
import evaluate
from datasets import load_dataset
from tqdm import tqdm
import re
import locale
from transformers import AutoTokenizer, AutoModelForCausalLM

os.environ["TOKENIZERS_PARALLELISM"] = "false"
locale.getpreferredencoding = lambda: "UTF-8"
# os.envion['LC_ALL'] = 'en_US.UTF-8'

In [3]:
# You will load this model in a 4-bit inference mode to optimize resource usage.
# This setup involves implementing a class named LLM that facilitates loading the model
# and generating text completions based on provided prompts
class LLM(object):
    def __init__(self, model_name="codellama/CodeLlama-7b-hf"):
        self.model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
        # pass

    def generate(self, prompts: List[str], **kwargs) -> List[str]:
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
            self.model.resize_token_embeddings(len(self.tokenizer))
        modelInput = self.tokenizer(
            prompts, return_tensors="pt", padding=True
        ).to("cuda")
        # self.tokenizer(prompts, return_tensors="pt", padding=True).to("cuda")

        return self.model.generate(**modelInput, max_new_tokens=100)
        pass

In [6]:
llm = LLM()

generated_ids = llm.generate(["A list of colors: red, blue", "Portugal is"])
llm.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
# type((List(["A list of colors: red, blue", "Portugal is"])))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


['A list of colors: red, blue, green, yellow, orange, purple, brown, pink, black, white, gray, brown, orange, purple, brown, pink, black, white, gray, brown, orange, purple, brown, pink, black, white, gray, brown, orange, purple, brown, pink, black, white, gray, brown, orange, purple, brown, pink, black, white, gray, brown, orange, purple,',
 'Portugal is a small town in the state of New York, United States. \n\n## History\n\nThe town was formed in 1811 from part of the town of New Paltz.\n\n## Geography\n\nAccording to the United States Census Bureau, the town has a total area of 36.9 square miles (96 km²), of which 36.8 square miles (95 km²) is land and 0.1 square miles']

In [5]:
class Evaluator(ABC):
    def __init__(self, llm):
        self.llm = llm

    @abstractmethod
    def load_data(self):
        pass

    @abstractmethod
    def build_prompts(self):
        pass

    @abstractmethod
    def postprocess_output(self, output: str) -> str:
        pass

    def generate_completions(self, prompts: List[str], batch_size=4, **kwargs) -> List[str]:
        generated_completions = []

        for i in range(0, len(prompts), batch_size):
          batch_prompts = prompts[i:i+batch_size]
          batch_id = self.llm.generate(batch_prompts)
          for j in batch_id:
            generated_completions.append([self.llm.tokenizer.batch_decode(batch_id)])

        return generated_completions
        # raise NotImplementedError

    def evaluate(self, evalset_file, batch_size=4, save_dir="outputs", max_new_tokens=128, **kwargs):
        dataset = self.load_data(evalset_file)
        prompts = self.build_prompts(dataset)
        outputs = self.generate_completions(prompts, batch_size=batch_size, max_new_tokens=max_new_tokens, **kwargs)

        predictions = []
        for i, (example, prompt, output) in enumerate(zip(dataset, prompts, outputs)):
            prediction = {
                "task_id": example.get("task_id", f"task_{i}"),
                "prompt": prompt,
                "completion": self.postprocess_output(output)
            }
            predictions.append(prediction)

        # Save predictions to file
        os.makedirs(save_dir, exist_ok=True)
        prediction_save_path = os.path.join(save_dir, f"{type(self).__name__}_predictions.jsonl")
        with open(prediction_save_path, "w") as fout:
            for pred in predictions:
                fout.write(json.dumps(pred) + "\n")

        # Calculate metrics and print results
        results = self.calculate_metrics(predictions, dataset)
        print(f"Results for {type(self).__name__}: {results}")

    @abstractmethod
    def calculate_metrics(self):
        pass

In [ ]:
# FOR TESTING ONLY
# Delete before submission

# Free GPU
from numba import cuda
device = cuda.get_current_device()
device.reset()

## Zero-shot Code Generation

In [ ]:
!mkdir -p human_eval
!wget -O human_eval/__init__.py https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A2/human_eval/__init__.py
!wget -O human_eval/data.py human_eval https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A2/human_eval/data.py
!wget -O human_eval/evaluation.py https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A2/human_eval/evaluation.py
!wget -O human_eval/execution.py human_eval https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A2/human_eval/execution.py

!mkdir -p data/humaneval
!wget -O data/humaneval/HumanEval.jsonl.gz https://github.com/openai/human-eval/raw/master/data/HumanEval.jsonl.gz

In [7]:
from human_eval.data import read_problems
from human_eval.evaluation import evaluate_functional_correctness

class HumanEvalEvaluator(Evaluator):
    def load_data(self, evalset_file="data/humaneval/HumanEval.jsonl.gz") -> List[Dict[str, Any]]:
        """
        Load the humaneval dataset
        :param evalset_file: path to the humaneval dataset file
        :return: list of examples
        """
        return list(read_problems(evalset_file).values())

    def build_prompts(self, dataset) -> List[str]:
        """
        Build zero-shot prompts from the humaneval dataset.
        """
        prompts = [example["prompt"] for example in dataset]
        return prompts

    def postprocess_output(self, output: str) -> str:
        stop_sequences=["\nclass", "\ndef", "\n#", "\nif", "\nprint"]

        for seq in stop_sequences:
            if seq in output:
                output = output.split(seq)[0]
                break

        return output.strip()
        # raise NotImplementedError

    def calculate_metrics(self, predictions, dataset):
        pass_at_k_results = evaluate_functional_correctness(
            sample_file=os.path.join("outputs", f"{type(self).__name__}_predictions.jsonl"),
            k=[1],
            problems={example["task_id"]: example for example in dataset},
            n_workers=64
        )
        return pass_at_k_results


ModuleNotFoundError: No module named 'human_eval'

In [ ]:
human_eval_evaluator = HumanEvalEvaluator(llm)
human_eval_evaluator.evaluate(evalset_file="data/humaneval/HumanEval.jsonl.gz", batch_size=16)

## Few-shot Math Reasoning

In [8]:
GSM_EXAMPLARS = [
    {
        "question": "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
        "cot_answer": "There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. So the answer is 6.",
        "pot_answer": "def solution():\n    \"\"\"There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\"\"\"\n    trees_initial = 15\n    trees_after = 21\n    trees_added = trees_after - trees_initial\n    result = trees_added\n    return result",
        "short_answer": "6"
    },
    {
        "question": "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
        "cot_answer": "There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. So the answer is 5.",
        "pot_answer": "def solution():\n    \"\"\"If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\"\"\"\n    cars_initial = 3\n    cars_arrived = 2\n    total_cars = cars_initial + cars_arrived\n    result = total_cars\n    return result",
        "short_answer": "5"
    },
    {
        "question": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
        "cot_answer": "Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. So the answer is 39.",
        "pot_answer": "def solution():\n    \"\"\"Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\"\"\"\n    leah_chocolates = 32\n    sister_chocolates = 42\n    total_chocolates = leah_chocolates + sister_chocolates\n    chocolates_eaten = 35\n    chocolates_left = total_chocolates - chocolates_eaten\n    result = chocolates_left\n    return result",
        "short_answer": "39"
    },
    {
        "question": "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
        "cot_answer": "Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. So the answer is 8.",
        "pot_answer": "def solution():\n    \"\"\"Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?\"\"\"\n    jason_lollipops_initial = 20\n    jason_lollipops_after = 12\n    denny_lollipops = jason_lollipops_initial - jason_lollipops_after\n    result = denny_lollipops\n    return result",
        "short_answer": "8"
    },
    {
        "question": "Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?",
        "cot_answer": "Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. So the answer is 9.",
        "pot_answer": "def solution():\n    \"\"\"Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\"\"\"\n    toys_initial = 5\n    mom_toys = 2\n    dad_toys = 2\n    total_received = mom_toys + dad_toys\n    total_toys = toys_initial + total_received\n    result = total_toys\n    return result",
        "short_answer": "9"
    },
    {
        "question": "There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?",
        "cot_answer": "There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. So the answer is 29.",
        "pot_answer": "def solution():\n    \"\"\"Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\"\"\"\n    toys_initial = 5\n    mom_toys = 2\n    dad_toys = 2\n    total_received = mom_toys + dad_toys\n    total_toys = toys_initial + total_received\n    result = total_toys\n    return result",
        "short_answer": "29"
    },
    {
        "question": "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?",
        "cot_answer": "Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. So the answer is 33.",
        "pot_answer": "def solution():\n    \"\"\"Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\"\"\"\n    golf_balls_initial = 58\n    golf_balls_lost_tuesday = 23\n    golf_balls_lost_wednesday = 2\n    golf_balls_left = golf_balls_initial - golf_balls_lost_tuesday - golf_balls_lost_wednesday\n    result = golf_balls_left\n    return result",
        "short_answer": "33"
    },
    {
        "question": "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?",
        "cot_answer": "Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. So the answer is 8.",
        "pot_answer": "def solution():\n    \"\"\"Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\"\"\"\n    money_initial = 23\n    bagels = 5\n    bagel_cost = 3\n    money_spent = bagels * bagel_cost\n    money_left = money_initial - money_spent\n    result = money_left\n    return result",
        "short_answer": "8"
    }
]

In [46]:
from sklearn.metrics import accuracy_score

class GSM8KEvaluator(Evaluator):
    def load_data(self, evalset_file="gsm8k") -> List[Dict[str, Any]]:
        """
        Load the GSM8K dataset https://huggingface.co/datasets/gsm8k with Huggingface datasets library
        Load the first 100 examples from the test split in main subset.
        """
        # dataset = load_dataset("https://huggingface.co/datasets/gsm8k", split="test", )
        dataset = dataset = load_dataset("gsm8k", 'main', split="test")

        return dataset[:100]
        # raise NotImplementedError

    def build_prompts(self, dataset, n_shot=8, demos=GSM_EXAMPLARS):
        """
        Build few-shot prompts from the GSM8K dataset. Use
        :param dataset: list of examples
        :param n_shot: number of examples to use for few-shot learning
        :param demos: list of demonstrator examples
        :return: list of prompts
        """
        # prompts = [example["prompt"] for example in dataset]
        prompts = []
        prompt = "Answer the following questions.\n"
        for demo in demos:
            prompt+="Question: " + demo["question"] + "\n"
            prompt+="Answer: " + demo["short_answer"] + "\n"

        # print("dataset ", dataset)
        # print("dataset["question"]", dataset["question"])
        # print("dataset["answer"]", dataset["answer"])
        for example in dataset["question"]:
            # print("example", example)
            prompts.append(prompt + "Question: " + example + "\n")

        print("prompts", prompts)
        return prompts
        # raise NotImplementedError

    def postprocess_output(self, output: str) -> str:
        """
        Postprocess the output from the language model.
        """
        stop_sequences=["\nclass", "\ndef", "\n#", "\nif", "\nprint"]

        for seq in stop_sequences:
            if seq in output:
                output = output.split(seq)[0]
                break

        return output.strip()
        # raise NotImplementedError

    def calculate_metrics(self, predictions, dataset):
        """
        Calculate accuracy metrics for the GSM8K dataset
        """
        accuracy = accuracy_score([example for example in dataset["answer"]], [prediction["completion"] for prediction in predictions])
        # accuracy = 0
        return accuracy
        # raise NotImplementedError

In [47]:
gsm8k_evaluator = GSM8KEvaluator(llm)
gsm8k_evaluator.evaluate(evalset_file="gsm8k")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


prompts ["Answer the following questions.\nQuestion: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nAnswer: 6\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nAnswer: 5\nQuestion: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\nAnswer: 39\nQuestion: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?\nAnswer: 8\nQuestion: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\nAnswer: 9\nQuestion: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?\nAnswer: 29\nQuestion: Michael had 58 golf balls. 

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.09 GiB is free. Process 2029 has 13.65 GiB memory in use. Of the allocated memory 10.39 GiB is allocated by PyTorch, and 3.14 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Few-shot Chain-of Thought Math Reasoning

In [ ]:

from sklearn.metrics import accuracy_score

class GSM8KCoTEvaluator(GSM8KEvaluator):
    def load_data(self, evalset_file="gsm8k") -> List[Dict[str, Any]]:
        """
        Load the GSM8K dataset https://huggingface.co/datasets/gsm8k with Huggingface datasets library
        Load the first 100 examples from the test split in main subset.
        """
        dataset = load_dataset("gsm8k", split="test", data_dir=evalset_file)
        return dataset[:100]
        # raise NotImplementedError

    def build_prompts(self, dataset, n_shot=8, demos=GSM_EXAMPLARS):
        """
        Build few-shot prompts from the GSM8K dataset. Use
        :param dataset: list of examples
        :param n_shot: number of examples to use for few-shot learning
        :param demos: list of demonstrator examples
        :return: list of prompts
        """
        # prompts = [example["prompt"] for example in dataset]
        prompts = []
        prompt = "Answer the following questions.\n"
        for demo in demos:
            prompt.append("Question: " + demo["question"] + "\n")
            prompt.append("Answer: " + demo["cot_answer"] + "\n")

        for example in dataset:
            prompts.append(prompt + "Question: " + example["question"] + "\n")

        return prompts
        # raise NotImplementedError

    def postprocess_output(self, output: str) -> str:
        """
        Postprocess the output from the language model.
        """
        stop_sequences=["\nclass", "\ndef", "\n#", "\nif", "\nprint"]

        for seq in stop_sequences:
            if seq in output:
                output = output.split(seq)[0]
                break

        return output.strip()
        # raise NotImplementedError

    def calculate_metrics(self, predictions, dataset):
        """
        Calculate accuracy metrics for the GSM8K dataset
        """
        accuracy = accuracy_score([example["cot_answer"] for example in dataset], [prediction["completion"] for prediction in predictions])

        return accuracy

In [ ]:
gsm8k_cot_evaluator = GSM8KCoTEvaluator(llm)
gsm8k_cot_evaluator.evaluate()

## Few-shot Program-of Thought Math Reasoning

In [ ]:
!pip install timeout-decorator Pebble
!wget -O python_executor.py https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A2/python_executor.py

In [ ]:
from python_executor import PythonExecutor
executor = PythonExecutor(get_answer_expr='solution()')

codes = [
    "def solution():\n    return 1 + 1",
    "def solution():\n    return 2 * 2",
]

predictions = []
runtime_errors = []
for pred, err in executor.batch_apply(codes):
    predictions.append(str(pred))
    runtime_errors.append(str(err['exec_info']).strip())

In [ ]:
predictions

In [ ]:
from sklearn.metrics import accuracy_score
class GSM8KPoTEvaluator(Evaluator):
    def load_data(self, evalset_file="gsm8k") -> List[Dict[str, Any]]:
        """
        Load the GSM8K dataset https://huggingface.co/datasets/gsm8k with Huggingface datasets library
        Load the first 100 examples from the test split in main subset.
        """
        dataset = load_dataset("gsm8k", split="test", data_dir=evalset_file)
        return dataset[:100]
        # raise NotImplementedError

    def build_prompts(self, dataset, n_shot=8, demos=GSM_EXAMPLARS):
        """
        Build few-shot prompts from the GSM8K dataset. Use
        :param dataset: list of examples
        :param n_shot: number of examples to use for few-shot learning
        :param demos: list of demonstrator examples
        :return: list of prompts
        """
        # prompts = [example["prompt"] for example in dataset]
        prompts = []
        prompt = "Answer the following questions.\n"
        for demo in demos:
            prompt.append("Question: " + demo["question"] + "\n")
            prompt.append("Answer: " + demo["pot_answer"] + "\n")

        for example in dataset:
            prompts.append(prompt + "Question: " + example["question"] + "\n")

        return prompts
        # raise NotImplementedError

    def postprocess_output(self, output: str) -> str:
        """
        Postprocess the output from the language model.
        """
        stop_sequences=["\nclass", "\ndef", "\n#", "\nif", "\nprint"]

        for seq in stop_sequences:
            if seq in output:
                output = output.split(seq)[0]
                break

        return output.strip()
        # raise NotImplementedError

    def calculate_metrics(self, predictions, dataset):
        """
        Calculate accuracy metrics for the GSM8K dataset
        """
        accuracy = accuracy_score([example["pot_answer"] for example in dataset], [prediction["completion"] for prediction in predictions])

        return accuracy

|                    | GSM8K |
|--------------------|-------|
| Direct Prompting   |       |
| Chain-of-Thought   |       |
| Program-of-Thought |       |